In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/FINAL.csv")

# Drop the 'review_comment_title' column
df.drop(columns=['review_comment_title'], inplace=True)

# Print confirmation and preview
print("✅ 'review_comment_title' column removed successfully.\n")
print("🔍 Preview of dataset after column removal:\n")
print(df.head())

✅ 'review_comment_title' column removed successfully.

🔍 Preview of dataset after column removal:

   Unnamed: 0 customer_city     seller_city seller_state  \
0           0     sao paulo            maua           SP   
1           1     sao paulo            maua           SP   
2           2     sao paulo            maua           SP   
3           3     barreiras  belo horizonte           SP   
4           4    vianopolis         guariba           SP   

  product_category_name_english   price  payment_value payment_type  \
0                    housewares   29.99          18.12  credit_card   
1                    housewares   29.99           2.00      voucher   
2                    housewares   29.99          18.59      voucher   
3                     perfumery  118.70         141.46       boleto   
4                          auto  159.90         179.12  credit_card   

  order_status order_purchase_timestamp order_delivered_customer_date  \
0    delivered          10/2/2017 10:56 

In [ ]:
# Identify missing values
print("\n📊 Missing values per column:\n")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Print total missing values
print("\n🧮 Total missing values in the dataset:", missing_values.sum())


📊 Missing values per column:

seller_city                        833
seller_state                       833
product_category_name_english     2567
price                              833
payment_value                        3
payment_type                         3
order_delivered_customer_date     3421
review_score                       997
review_comment_message           69062
review_creation_date               997
dtype: int64

🧮 Total missing values in the dataset: 79549


In [ ]:
# Remove rows with missing 'review_comment_message'
df.dropna(subset=['review_comment_message'], inplace=True)

# Print confirmation
print("✅ Rows with missing 'review_comment_message' have been removed.")

# Check if any missing values remain
print("\n📊 Missing values per column after cleanup:\n")
missing_after = df.isnull().sum()
print(missing_after[missing_after > 0])

# Print total remaining missing values
print("\n🧮 Total remaining missing values in the dataset:", missing_after.sum())


✅ Rows with missing 'review_comment_message' have been removed.

📊 Missing values per column after cleanup:

seller_city                       563
seller_state                      563
product_category_name_english    1332
price                             563
payment_value                       3
payment_type                        3
order_delivered_customer_date    2144
dtype: int64

🧮 Total remaining missing values in the dataset: 5171


In [ ]:
# Remove rows with missing values in specified columns
df.dropna(subset=[
    'seller_city',
    'seller_state',
    'product_category_name_english',
    'payment_value'
], inplace=True)

# Print confirmation
print("✅ Rows with missing values in 'seller_city', 'seller_state', 'product_category_name_english', and 'payment_value' have been removed.")

# Show remaining missing values
print("\n📊 Missing values per column after cleanup:\n")
remaining_missing = df.isnull().sum()
print(remaining_missing[remaining_missing > 0])

# Print total remaining missing values
print("\n🧮 Total remaining missing values in the dataset:", remaining_missing.sum())


✅ Rows with missing values in 'seller_city', 'seller_state', 'product_category_name_english', and 'payment_value' have been removed.

📊 Missing values per column after cleanup:

order_delivered_customer_date    1535
dtype: int64

🧮 Total remaining missing values in the dataset: 1535


In [ ]:
# Convert to datetime if not already
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Fill missing values with the median date
median_date = df['order_delivered_customer_date'].median()
df['order_delivered_customer_date'].fillna(median_date, inplace=True)

# Confirmation
print(f"✅ Filled missing 'order_delivered_customer_date' values with median date: {median_date.date()}")

# Final missing check
print("\n📊 Final check for missing values:\n")
final_missing = df.isnull().sum()
print(final_missing[final_missing > 0] if final_missing.sum() > 0 else "🎉 No missing values remaining!")



✅ Filled missing 'order_delivered_customer_date' values with median date: 2018-01-31

📊 Final check for missing values:

🎉 No missing values remaining!


<ipython-input-24-18e9c14034ab>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['order_delivered_customer_date'].fillna(median_date, inplace=True)


In [ ]:
# Step 1: Identify duplicates
duplicate_rows = df[df.duplicated()]
print(f"🔍 Number of duplicate rows found: {len(duplicate_rows)}")


print(f"\n📐 Final dataset shape: {df.shape}")


🔍 Number of duplicate rows found: 0

📐 Final dataset shape: (48746, 14)


For more clear output code, for outliers, IQR

In [ ]:


# Step 1: Detecting Outliers using IQR method
# Define function to detect outliers
def detect_outliers(df, column):
    # Calculate the IQR for each column
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers, lower_bound, upper_bound

# Loop through numeric columns to detect outliers
outliers_dict = {}
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    outliers, lower, upper = detect_outliers(df, column)
    if not outliers.empty:
        outliers_dict[column] = {
            "outliers": outliers,
            "lower_bound": lower,
            "upper_bound": upper
        }

# Step 2: Print outliers found with clear output
if outliers_dict:
    print("Outliers detected in the following columns:\n")
    for column, data in outliers_dict.items():
        print(f"\nColumn: {column}")
        print(f"Lower Bound: {data['lower_bound']}")
        print(f"Upper Bound: {data['upper_bound']}")
        print(f"Outliers detected:\n{data['outliers']}\n")
else:
    print("No outliers detected.")


Outliers detected in the following columns:


Column: price
Lower Bound: 9.5
Upper Bound: 9.5
Outliers detected:
        Unnamed: 0        customer_city     seller_city seller_state  \
724            724       belo horizonte       sao paulo           SP   
725            725       belo horizonte       sao paulo           SP   
727            727       belo horizonte       sao paulo           SP   
1162          1162            sao paulo       sao paulo           SP   
2384          2384         porto alegre       sao paulo           SP   
...            ...                  ...             ...          ...   
117348      117348            sao paulo        louveira           SP   
118161      118161      mogi das cruzes       sao paulo           SP   
118162      118162      mogi das cruzes       sao paulo           SP   
118543      118543  americo brasiliense  ribeirao preto           SP   
118919      118919             salvador       aparecida           SP   

          product_cate

In [ ]:
# Step 1: Detecting Outliers using IQR method
# Define function to detect outliers
def detect_outliers(df, column):
    # Calculate the IQR for each column
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers, lower_bound, upper_bound

# Loop through numeric columns to detect outliers
outliers_dict = {}
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    outliers, lower, upper = detect_outliers(df, column)
    if not outliers.empty:
        outliers_dict[column] = len(outliers)  # Store the count of outliers

# Step 2: Print concise outliers found
if outliers_dict:
    print("Outliers detected in the following columns:")
    for column, outlier_count in outliers_dict.items():
        print(f"Column: {column}, Outliers Count: {outlier_count}")
else:
    print("No outliers detected.")


Outliers detected in the following columns:
Column: price, Outliers Count: 402
Column: payment_value, Outliers Count: 543


Function to handle outliers

In [ ]:
import pandas as pd

# Step 1: Detecting Outliers using IQR method
def detect_outliers(df, column):
    # Calculate the IQR for each column
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers, lower_bound, upper_bound

# Function to handle outliers
def handle_outliers(df, column, method='remove'):
    outliers, lower, upper = detect_outliers(df, column)

    if method == 'remove':
        # Remove rows containing outliers
        df_cleaned = df[~df[column].isin(outliers[column])]

    elif method == 'cap':
        # Cap outliers at lower and upper bounds
        df_cleaned = df.copy()
        df_cleaned[column] = df[column].apply(lambda x: max(min(x, upper), lower) if x < lower or x > upper else x)

    elif method == 'impute':
        # Impute outliers with the median value
        median_value = df[column].median()
        df_cleaned = df.copy()
        df_cleaned[column] = df[column].apply(lambda x: median_value if x < lower or x > upper else x)

    return df_cleaned

# Loop through numeric columns to detect and handle outliers
df_cleaned = df.copy()  # Make a copy of the dataframe to apply changes

# Dictionary to store outliers before handling
outliers_before = {}

# Detect and store outliers before handling
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    outliers, lower, upper = detect_outliers(df, column)
    if not outliers.empty:
        outliers_before[column] = outliers

# Handle outliers (method 'remove' or 'cap' or 'impute')
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    df_cleaned = handle_outliers(df_cleaned, column, method='impute')  # You can change method to 'cap' or 'impute'

# Dictionary to store outliers after handling
outliers_after = {}

# Detect and store outliers after handling
for column in df_cleaned.select_dtypes(include=['float64', 'int64']).columns:
    outliers, lower, upper = detect_outliers(df_cleaned, column)
    if not outliers.empty:
        outliers_after[column] = outliers

# Output the results
print("\nOutliers detected before handling:")
for column, outliers in outliers_before.items():
    print(f"\nColumn: {column}")
    print(f"Outliers before handling:\n{outliers}")
    print(f"Number of outliers before handling: {len(outliers)}")

print("\nOutliers detected after handling:")
for column, outliers in outliers_after.items():
    print(f"\nColumn: {column}")
    print(f"Outliers after handling:\n{outliers}")
    print(f"Number of outliers after handling: {len(outliers)}")

if not outliers_after:
    print("\nNo outliers remain after handling.")


# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('preprocessedFinal.csv', index=False)
print("\nData has been saved to 'preprocessedFinal.csv'.")


Outliers detected before handling:

Column: price
Outliers before handling:
        Unnamed: 0        customer_city     seller_city seller_state  \
724            724       belo horizonte       sao paulo           SP   
725            725       belo horizonte       sao paulo           SP   
727            727       belo horizonte       sao paulo           SP   
1162          1162            sao paulo       sao paulo           SP   
2384          2384         porto alegre       sao paulo           SP   
...            ...                  ...             ...          ...   
117348      117348            sao paulo        louveira           SP   
118161      118161      mogi das cruzes       sao paulo           SP   
118162      118162      mogi das cruzes       sao paulo           SP   
118543      118543  americo brasiliense  ribeirao preto           SP   
118919      118919             salvador       aparecida           SP   

          product_category_name_english  price  payment_va

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/preprocessedFinal.csv")

print(df.shape)
# Step 1: Detecting Outliers using IQR method
# Define function to detect outliers
def detect_outliers(df, column):
    # Calculate the IQR for each column
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers, lower_bound, upper_bound

# Loop through numeric columns to detect outliers
outliers_dict = {}
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    outliers, lower, upper = detect_outliers(df, column)
    if not outliers.empty:
        outliers_dict[column] = len(outliers)  # Store the count of outliers

# Step 2: Print concise outliers found
if outliers_dict:
    print("Outliers detected in the following columns:")
    for column, outlier_count in outliers_dict.items():
        print(f"Column: {column}, Outliers Count: {outlier_count}")
else:
    print("No outliers detected.")

# Final missing check
print("\n📊 Final check for missing values:\n")
final_missing = df.isnull().sum()
print(final_missing[final_missing > 0] if final_missing.sum() > 0 else "🎉 No missing values remaining!")


(48746, 14)
No outliers detected.

📊 Final check for missing values:

🎉 No missing values remaining!
